### Initialize

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-nazrerukmini')

### Generate TIF (NDVI)

In [ ]:
# Function to process a single point
def process_point(lon, lat, city, state, year, filename):
    # Define the center point
    center = ee.Geometry.Point([lon, lat])

    # Define the bounding box with a buffer radius (2560 meters for 512x512 pixels at 10m resolution)
    geometry = center.buffer(2560).bounds()

    # Load Sentinel-2 MSI Level-2A as ImageCollection
    sentinel2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                 .filterBounds(geometry)
                 .filterDate(f'{year}-07-01', f'{year}-09-30')
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 3))
                 .median())

    # Calculate NDVI
    ndvi = sentinel2.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Classify NDVI into three categories
    ndvi_classified = ndvi.expression(
        "(ndvi < 0.4) ? 1 : (ndvi <= 0.7) ? 2 : 3",
        {'ndvi': ndvi}
    ).rename('NDVI_Classified')

    # Visualization parameters for the classified NDVI
    ndvi_class_vis = {
        'min': 1,
        'max': 3,
        'palette': ['white', 'lightgreen', 'darkgreen']  # Assign colors to each class
    }

    # Enhance each band by a different factor
    enhanced_rgb = (sentinel2.select(['B4', 'B3', 'B2']).multiply(2.0))

    viz_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1
    }

    # Export EnhancedRGB Layer to Google Drive
    enhanced_rgb_task = ee.batch.Export.image.toDrive(
        image=enhanced_rgb.clip(geometry).visualize(**viz_params),
        description=f'EnhancedRGB_{filename}_{year}',
        #folder=year,
        fileNamePrefix=f'{filename}_{year}',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512'
    )
    enhanced_rgb_task.start()

    # Export the classified NDVI as a single-band image
    ndvi_classified_task = ee.batch.Export.image.toDrive(
        image=ndvi_classified.clip(geometry),
        description=f'NDVI_Classified_{filename}_{year}',
        #folder=year,
        fileNamePrefix=f'm_NDVI_{filename}_{year}',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512'
    )
    ndvi_classified_task.start()

    print(f"Tasks started for {city} at ({lon}, {lat}).")


##COMBINED RGB + MASK SVM

In [ ]:
def process_point_2(lon, lat, city, state, year, filename):
    # Define the center point
    center = ee.Geometry.Point([lon, lat])

    # Define the bounding box with a buffer radius (2560 meters for 512x512 pixels at 10m resolution)
    geometry = center.buffer(2560).bounds()

    # Load Sentinel-2 MSI Level-2A as ImageCollection
    sentinel2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                 .filterBounds(geometry)
                 .filterDate(f'{year}-07-01', f'{year}-09-30')
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 3))
                 .median())

    # Calculate indices and masks
    ndvi = sentinel2.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = sentinel2.normalizedDifference(['B3', 'B8']).rename('NDWI')
    swm = sentinel2.expression(
        '(B2 + B3) / (B8 + B11)',
        {
            'B2': sentinel2.select('B2'),
            'B3': sentinel2.select('B3'),
            'B8': sentinel2.select('B8'),
            'B11': sentinel2.select('B11')
        }
    ).rename('SWM')

    # Combine NDVI and NDWI into a single image for sampling
    feature_image = ndvi.addBands(ndwi)

    # Define class masks
    water_mask = swm.gt(1.5)
    building_mask = ndvi.gt(0.0).And(ndvi.lt(0.4))
    sparse_veg_mask = ndvi.gte(0.4).And(ndvi.lt(0.7))
    dense_veg_mask = ndvi.gte(0.7)

    # Sample points for each class
    NUM_OF_PIXELS = 50000

    def sample_points(mask, class_value):
        return feature_image.updateMask(mask).sample(
            region=geometry,
            scale=10,
            numPixels=NUM_OF_PIXELS,
            geometries=True
        ).map(lambda f: f.set('class', class_value))

    water_points = sample_points(water_mask, 0)
    building_points = sample_points(building_mask, 1)
    sparse_veg_points = sample_points(sparse_veg_mask, 2)
    dense_veg_points = sample_points(dense_veg_mask, 3)

    # Merge all training points
    training_points = water_points.merge(building_points).merge(sparse_veg_points).merge(dense_veg_points)

    # Split the data into training (80%) and validation (20%) sets
    with_random = training_points.randomColumn()
    split = 0.8
    training_data = with_random.filter(ee.Filter.lt('random', split))
    validation_data = with_random.filter(ee.Filter.gte('random', split))

    # Train an SVM classifier
    svm_classifier = ee.Classifier.libsvm().train(
        features=training_data,
        classProperty='class',
        inputProperties=['NDVI', 'NDWI']
    )

    # Classify the image
    classified_image = feature_image.classify(svm_classifier)

    # Evaluate the classifier using the validation data
    validation = validation_data.classify(svm_classifier)
    validation_confusion_matrix = validation.errorMatrix('class', 'classification')
    print('Validation Error Matrix:', validation_confusion_matrix.getInfo())
    print('Validation Overall Accuracy:', validation_confusion_matrix.accuracy().getInfo())

    # Compute training accuracy
    training_confusion_matrix = svm_classifier.confusionMatrix()
    print('Training Error Matrix:', training_confusion_matrix.getInfo())
    print('Training Overall Accuracy:', training_confusion_matrix.accuracy().getInfo())

    # Enhanced RGB export
    enhanced_rgb = sentinel2.select(['B4', 'B3', 'B2']).multiply(2.0)
    viz_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1
    }

    # Export tasks
    # 1. Enhanced RGB
    enhanced_rgb_task = ee.batch.Export.image.toDrive(
        image=enhanced_rgb.clip(geometry).visualize(**viz_params),
        description=f'EnhancedRGB_{filename}_{year}',
        folder='ChangeDetection',
        fileNamePrefix=f'{filename}_{year}',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512'
    )

    # 2. Classified Image
    class_vis = {
        'min': 0,
        'max': 3,
        'palette': ['lightblue', 'white', 'lightgreen', 'darkgreen']
    }
    mask_task = ee.batch.Export.image.toDrive(
        image=classified_image,
        description=f'm_{filename}_{year}',
        folder='ChangeDetection',
        fileNamePrefix=f'm_{filename}_{year}',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512'
    )

    # Start both export tasks
    enhanced_rgb_task.start()
    mask_task.start()

    print(f"Tasks started for {city} at ({lon}, {lat}).")

### Data

### RUN

In [ ]:
import io
import pandas as pd
data = pd.read_csv(io.StringIO('''
California,SanFrancisco,2019,2024,"-122.447559212022,37.7587570417743",37.75875704,-122.4475592,"-122.447559212022,37.8087570417743","-122.397559212022,37.8087570417743","-122.397559212022,37.7587570417743","-122.397559212022,37.7087570417743","-122.447559212022,37.7087570417743","-122.497559212022,37.7087570417743","-122.497559212022,37.7587570417743","-122.497559212022,37.8087570417743"
'''), header=None)


In [ ]:
coord_list = [4, 7, 8, 9, 10, 11, 12, 13, 14]
coord_name = ['C', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
state = data[0][0]
ncities = data.shape[0]
# Process each city
for j in range(ncities):
  cityData = data.iloc[j]
  city = cityData[1]
  print(city, state)
  for i in range(9):
      coord = cityData.iloc[coord_list[i]].split(',')
      filename = f'{state}_{city}_{coord_name[i]}'
      print(filename, end=';')

      lon = float(coord[0])
      lat = float(coord[1])
      print(lon, lat)
      # process_point(lon, lat, city, state, '2019', filename)
      # process_point(lon, lat, city, state, '2024', filename)
      process_point_2(lon, lat, city, state, '2019', filename)
      process_point_2(lon, lat, city, state, '2024', filename)
  print()

print("All tasks have been started.")

SanFrancisco California
California_SanFrancisco_C;-122.447559212022 37.7587570417743
Validation Error Matrix: [[0, 1, 0, 0], [0, 6948, 20, 0], [0, 44, 1863, 21], [0, 0, 19, 916]]
Validation Overall Accuracy: 0.9893205858421481
Training Error Matrix: [[0, 2, 0, 0], [0, 28039, 52, 0], [0, 128, 7358, 74], [0, 0, 77, 3492]]
Training Overall Accuracy: 0.9915098669114273
Tasks started for SanFrancisco at (-122.447559212022, 37.7587570417743).
Validation Error Matrix: [[0, 1, 0, 0], [0, 6840, 21, 0], [0, 25, 1824, 11], [0, 0, 15, 1096]]
Validation Overall Accuracy: 0.9925760195260857
Training Error Matrix: [[0, 0, 0, 0], [0, 27239, 99, 0], [0, 94, 7327, 69], [0, 0, 59, 4248]]
Training Overall Accuracy: 0.9917976236105788
Tasks started for SanFrancisco at (-122.447559212022, 37.7587570417743).
California_SanFrancisco_N;-122.447559212022 37.8087570417743
Validation Error Matrix: [[5261, 1, 0, 0], [0, 2922, 15, 0], [0, 16, 876, 24], [0, 0, 6, 750]]
Validation Overall Accuracy: 0.9937189747745923

Validation Error Matrix: [[3485, 10, 0, 0], [2, 4508, 6, 0], [0, 5, 1074, 24], [0, 0, 7, 838]]
Validation Overall Accuracy: 0.9945777688522944
Training Error Matrix: [[13462, 32, 0, 0], [15, 17569, 27, 0], [0, 31, 4310, 98], [0, 0, 26, 3422]]
Training Overall Accuracy: 0.9941270004103406


### The Code Not Taken

#### Single City

In [ ]:
import io
import pandas as pd
data = pd.read_csv(io.StringIO('''
SanJose,2019,2024,"-121.899180305122,37.3391025913052",37.33910259,-121.8991803,"-121.899180305122,37.3891025913052","-121.849180305122,37.3891025913052","-121.849180305122,37.3391025913052","-121.849180305122,37.2891025913052","-121.899180305122,37.2891025913052","-121.949180305122,37.2891025913052","-121.949180305122,37.3391025913052","-121.949180305122,37.3891025913052"
'''), header=None)


In [ ]:
coord_list = [3, 6, 7, 8, 9, 10, 11, 12, 13]
coord_name = ['C', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
city='SanJose'
# Process each point
for i in range(9):
    coord = data.iloc[0, coord_list[i]].split(',')
    filename = f'{city}_{coord_name[i]}'
    lon = float(coord[0])
    lat = float(coord[1])

    print(filename)
    print(lon, lat)
    process_point(lon, lat, city, '2019', filename)

print("All tasks have been started.")


SanJose_C
-121.899180305122 37.3391025913052
Tasks started for SanJose at (-121.899180305122, 37.3391025913052).
SanJose_N
-121.899180305122 37.3891025913052
Tasks started for SanJose at (-121.899180305122, 37.3891025913052).
SanJose_NE
-121.849180305122 37.3891025913052
Tasks started for SanJose at (-121.849180305122, 37.3891025913052).
SanJose_E
-121.849180305122 37.3391025913052
Tasks started for SanJose at (-121.849180305122, 37.3391025913052).
SanJose_SE
-121.849180305122 37.2891025913052
Tasks started for SanJose at (-121.849180305122, 37.2891025913052).
SanJose_S
-121.899180305122 37.2891025913052
Tasks started for SanJose at (-121.899180305122, 37.2891025913052).
SanJose_SW
-121.949180305122 37.2891025913052
Tasks started for SanJose at (-121.949180305122, 37.2891025913052).
SanJose_W
-121.949180305122 37.3391025913052
Tasks started for SanJose at (-121.949180305122, 37.3391025913052).
SanJose_NW
-121.949180305122 37.3891025913052
Tasks started for SanJose at (-121.949180305122

In [ ]:
# Define the center and geometry
center = ee.Geometry.Point([-122.50, 37.77])
geometry = center.buffer(2560).bounds()

# Load Sentinel-2 MSI Level-2A as image collection
sentinel2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
             .filterBounds(geometry)
             .filterDate('2019-07-01', '2019-09-30')
             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 3))
             .median())

# Calculate NDVI and NDWI
ndvi = sentinel2.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndwi = sentinel2.normalizedDifference(['B3', 'B8']).rename('NDWI')
swm = sentinel2.expression(
    '(B2 + B3) / (B8 + B11)',
    {
        'B2': sentinel2.select('B2'),
        'B3': sentinel2.select('B3'),
        'B8': sentinel2.select('B8'),
        'B11': sentinel2.select('B11')
    }
).rename('SWM')

# Combine NDVI and NDWI into a single image for sampling
feature_image = ndvi.addBands(ndwi)

# Define class masks
water_mask = swm.gt(1.5)
building_mask = ndvi.gt(0.0).And(ndvi.lt(0.4))
sparse_veg_mask = ndvi.gte(0.4).And(ndvi.lt(0.7))
dense_veg_mask = ndvi.gte(0.7)

# Sample points for each class
NUM_OF_PIXELS = 50000

def sample_points(mask, class_value):
    return feature_image.updateMask(mask).sample(
        region=geometry,
        scale=10,
        numPixels=NUM_OF_PIXELS,
        geometries=True
    ).map(lambda f: f.set('class', class_value))

water_points = sample_points(water_mask, 0)
building_points = sample_points(building_mask, 1)
sparse_veg_points = sample_points(sparse_veg_mask, 2)
dense_veg_points = sample_points(dense_veg_mask, 3)

# Merge all training points
training_points = water_points.merge(building_points).merge(sparse_veg_points).merge(dense_veg_points)

# Split the data into training (80%) and validation (20%) sets
with_random = training_points.randomColumn()
split = 0.8
training_data = with_random.filter(ee.Filter.lt('random', split))
validation_data = with_random.filter(ee.Filter.gte('random', split))

# Train an SVM classifier
svm_classifier = ee.Classifier.libsvm().train(
    features=training_data,
    classProperty='class',
    inputProperties=['NDVI', 'NDWI']
)

# Classify the image
classified_image = feature_image.classify(svm_classifier)

# Evaluate the classifier using the validation data
validation = validation_data.classify(svm_classifier)
validation_confusion_matrix = validation.errorMatrix('class', 'classification')
print('Validation Error Matrix:', validation_confusion_matrix.getInfo())
print('Validation Overall Accuracy:', validation_confusion_matrix.accuracy().getInfo())

# Compute training accuracy
training_confusion_matrix = svm_classifier.confusionMatrix()
print('Training Error Matrix:', training_confusion_matrix.getInfo())
print('Training Overall Accuracy:', training_confusion_matrix.accuracy().getInfo())

# Visualization (requires exporting or downloading for Colab visualization)
# Visualization parameters
class_vis = {
    'min': 0,
    'max': 3,
    'palette': ['lightblue', 'white', 'lightgreen', 'darkgreen']
}

# Export the classified image to Google Drive
export_task = ee.batch.Export.image.toDrive(
    image=classified_image,
    description='ClassifiedImage',
    folder='ChangeDetection',
    fileNamePrefix='classified_image_test',
    region=geometry.getInfo()['coordinates'],
    # scale=10,
    fileFormat='GEOTIFF',
    crs='EPSG:3857',
    dimensions='512x512'
)
export_task.start()


GPT Functionless SVM

In [ ]:
# Define the center and geometry
center = ee.Geometry.Point([-122.50, 37.77])
geometry = center.buffer(2560).bounds()

# Load Sentinel-2 MSI Level-2A as image collection
sentinel2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
             .filterBounds(geometry)
             .filterDate('2019-07-01', '2019-09-30')
             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 3))
             .median())

# Calculate NDVI and NDWI
ndvi = sentinel2.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndwi = sentinel2.normalizedDifference(['B3', 'B8']).rename('NDWI')
swm = sentinel2.expression(
    '(B2 + B3) / (B8 + B11)',
    {
        'B2': sentinel2.select('B2'),
        'B3': sentinel2.select('B3'),
        'B8': sentinel2.select('B8'),
        'B11': sentinel2.select('B11')
    }
).rename('SWM')

# Combine NDVI and NDWI into a single image for sampling
feature_image = ndvi.addBands(ndwi)

# Define class masks
water_mask = swm.gt(1.5)
building_mask = ndvi.gt(0.0).And(ndvi.lt(0.4))
sparse_veg_mask = ndvi.gte(0.4).And(ndvi.lt(0.7))
dense_veg_mask = ndvi.gte(0.7)

# Sample points for each class
NUM_OF_PIXELS = 50000

def sample_points(mask, class_value):
    return feature_image.updateMask(mask).sample(
        region=geometry,
        scale=10,
        numPixels=NUM_OF_PIXELS,
        geometries=True
    ).map(lambda f: f.set('class', class_value))

water_points = sample_points(water_mask, 0)
building_points = sample_points(building_mask, 1)
sparse_veg_points = sample_points(sparse_veg_mask, 2)
dense_veg_points = sample_points(dense_veg_mask, 3)

# Merge all training points
training_points = water_points.merge(building_points).merge(sparse_veg_points).merge(dense_veg_points)

# Split the data into training (80%) and validation (20%) sets
with_random = training_points.randomColumn()
split = 0.8
training_data = with_random.filter(ee.Filter.lt('random', split))
validation_data = with_random.filter(ee.Filter.gte('random', split))

# Train an SVM classifier
svm_classifier = ee.Classifier.libsvm().train(
    features=training_data,
    classProperty='class',
    inputProperties=['NDVI', 'NDWI']
)

# Classify the image
classified_image = feature_image.classify(svm_classifier)

# Evaluate the classifier using the validation data
validation = validation_data.classify(svm_classifier)
validation_confusion_matrix = validation.errorMatrix('class', 'classification')
print('Validation Error Matrix:', validation_confusion_matrix.getInfo())
print('Validation Overall Accuracy:', validation_confusion_matrix.accuracy().getInfo())

# Compute training accuracy
training_confusion_matrix = svm_classifier.confusionMatrix()
print('Training Error Matrix:', training_confusion_matrix.getInfo())
print('Training Overall Accuracy:', training_confusion_matrix.accuracy().getInfo())

# Visualization (requires exporting or downloading for Colab visualization)
# Visualization parameters
class_vis = {
    'min': 0,
    'max': 3,
    'palette': ['lightblue', 'white', 'lightgreen', 'darkgreen']
}

# Export the classified image to Google Drive
export_task = ee.batch.Export.image.toDrive(
    image=classified_image,
    description='ClassifiedImage',
    folder='ChangeDetection',
    fileNamePrefix='classified_image_test',
    region=geometry.getInfo()['coordinates'],
    # scale=10,
    fileFormat='GEOTIFF',
    crs='EPSG:3857',
    dimensions='512x512'
)
export_task.start()
